In [ ]:
import requests
from bs4 import BeautifulSoup
import jieba
import jieba.posseg as pseg
import time
import datetime
import random
import re
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

def is_Japanese(word):
    for ch in word:
        if '\u0800' <= ch <= '\u4e00':
            return True
    return False

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import numpy as np

!gdown --id "1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_"

Downloading...
From: https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
To: /content/TaipeiSansTCBeta-Regular.ttf
20.7MB [00:00, 78.2MB/s]


In [ ]:
#爬蟲
with open('singers_lyric.txt', 'w',encoding="utf-8") as f: 

    starttime = datetime.datetime.now()
    r1 = '[ a-zａ-ｚA-ZＡ-Ｚ0-9０-９’＾＆‧︵︶〃ˇşı﹡△〝〞═╗╚╝╰╯々\\\\×‥･：；︿ο∵×♫♫♬┌┐『』│─┘◎┌◇◆—－﹝﹞┐ⅣⅢⅡⅠ＜＞┌\─▎◥◤◣：◢╦▲→←♀♂ㄅㄆㄇㄈㄉㄊㄋㄌㄍㄎㄏㄐㄑㄒㄓㄔㄕㄖㄗㄘㄙㄧㄨㄩㄚㄛㄜㄝㄞㄟㄠㄡㄢㄣㄤㄥㄦ!"▽\˙↓ε⊙＃╭○○╮#$%&\'()（）*°﹒+,-./\:;<=>＊?@，。?★☆♥║╓●╥█\╔╩╬﹋גאולהסליחהתשובהॐमणिपद्मेहूँ＠｡◕‿◕｡、…【】「」《》？“”‘’！[\\]^_`．{|}⋯∼～~]+'
    singer=input('請輸入歌手名稱：')
    lyrics_lst = []
    links=[]
    for i in range(26):
        if is_Chinese(singer)==True: #如果輸入的歌手名字中有中文
            singer = re.sub(r1,'',singer) #就把輸入的東西刪掉只剩中文
            r = requests.get(f'https://www.kkbox.com/tw/tc/search.php?search=song&word={singer}&cur_page={i}')  #頁數(最後解決)
            soup = BeautifulSoup(r.text, 'html.parser')
            for body in soup.find_all('tbody'):
                if body.find_all('mark', {'class':'keyword c1'}) == []: #如果沒有關鍵字符合，先跳過(例如：四月天)
                    continue
                elif body.find('mark', {'class':'keyword c1'}).text == singer:
                    span = body.find_all('span')
                    if span !=[]:
                        for span in span:
                            span = span.text
                            r = re.sub(r1,'',span) #例如把kkbox中(五月天 Mayday)刪到只剩(五月天)
                            if r == singer:
                                link = body.find('a', {"class": "song-title"})
                                if link.text not in links:
                                    links.append(link)
                    else:
                        div = body.find('div',{'class':'song-artist-album'})
                        a = div.find_all('a')
                        singer_name = a[0].text
                        r = re.sub(r1,'',singer_name)
                        if r == singer:
                            link = body.find('a', {"class": "song-title"})
                            if link.text not in links:
                                links.append(link)
                # time.sleep(0.5)  #不知道要設多久，我先設0.5秒

        else:  #歌手名字非中文，那輸入的格式必須跟kkbox設定的一樣才有效
            r = requests.get(f'https://www.kkbox.com/tw/tc/search.php?search=song&word={singer}&cur_page={i}')  #頁數(最後解決)
            soup = BeautifulSoup(r.text, 'html.parser')
            for body in soup.find_all('tbody'):
                if body.find_all('mark', {'class':'keyword c1'}) == []:
                    continue
                elif body.find('mark', {'class':'keyword c1'}).text == singer:
                    span = body.find_all('span')
                    if span !=[]:
                        for span in span:
                            span = span.text
                            if span == singer:
                                link = body.find('a', {"class": "song-title"})
                                if link.text not in links:
                                    links.append(link)
                    else:
                        div = body.find('div',{'class':'song-artist-album'})
                        a = div.find_all('a')
                        singer_name = a[0].text
                        if singer_name == singer:
                            link = body.find('a', {"class": "song-title"})
                            if link.text not in links:
                                links.append(link)
                #time.sleep(0.5)  #不知道要設多久，我先設0.5秒 

    if links != []:
        for link in links:
            r2 = requests.get(f'https://www.kkbox.com{link.get("href")}')
            soup2 = BeautifulSoup(r2.text, 'html.parser')
            lyrics_ = soup2.find_all('p', {'class': 'lyrics'})
            lyrics = soup2.find('p', {'class': 'lyrics'})
            if lyrics_ ==[]: #有這首歌，但沒有歌詞
                continue
            else:        
                lyrics = lyrics.text.strip('\n')
                number = []
                for i in range(len(lyrics)): 
                    if lyrics[i]=='\n':
                        number.append(i) 
                    elif len(number)==20:
                        break
                if len(number)<5: 
                    continue 
                index_1 = number[2]
                index_2 = number[4]
                if lyrics.count('作詞')==2 and lyrics.count('作曲')==2 and lyrics.count('和聲演唱')==1:
                    index_3 = number[13]
                    lyrics = lyrics[index_3+1:] 
                elif lyrics.count('詞')==2 and lyrics.count('曲')==2 and lyrics.count('貝 斯 手')==1 and lyrics.count('鍵 盤 兼 中 提 琴 手')==1:
                    index_5 = number[16]
                    lyrics = lyrics[index_5+1:]
                elif lyrics.count('作詞')==2 and lyrics.count('作曲')==2 and lyrics.count('編曲')==2:
                    index_4 = number[10]
                    lyrics = lyrics[index_4+1:]
                elif '編曲' in lyrics:
                    lyrics = lyrics[index_2+1:]
                elif lyrics.count('作詞')==1 and lyrics.count('作曲')==1 and lyrics.count('男:')==2 and lyrics.count('女:')==2:
                    index_6 = number[6]
                    lyrics = lyrics[index_6+1:]
                elif lyrics.count('詞')==2 and lyrics.count('曲')==2:
                    lyrics = lyrics[index_2+1:]
                elif lyrics.count('作詞')==2 and lyrics.count('作曲')==2:
                    lyrics = lyrics[index_2+1:] 
                else:
                    lyrics = lyrics[index_1+1:]
                lyrics = re.sub(r1,'',lyrics) #把歌詞中一些奇怪的東西都刪掉
                japanese1 = re.compile(u"[\u30a0-\u30ff]+") 
                lyrics = japanese1.sub('',lyrics)
                japanese2 = re.compile(u"[\u3040-\u309f]+")
                lyrics = japanese2.sub('',lyrics)
                lyrics = lyrics.replace('\r','').replace('\ufeff','').replace('前奏','').replace('間奏中','').replace('間奏','').replace('開始囉','').replace('預備起','').replace('歌詞亂編虫','').replace('歌詞亂編','').replace('阿信','').replace('監製','').replace('尾奏','').replace('作詞','').replace('作曲','').replace('演唱','').replace('編曲','')
                lyrics = lyrics.replace('開始嚕','').replace('情歌唱完','').replace('休息一下吧','').replace('愛妳唷','').replace('最後一段囉','').replace('冠佑','').replace('瑪莎','').replace('休息一下','').replace('最愛五月天','').replace('演唱會阿信的感謝','').replace('它是機車','').replace('五月天','').replace('有很多是周杰倫更改歌詞','').replace('周杰倫','')
                lyrics = lyrics.replace('來囉','').replace('還有喔','').replace('很棒的吧哈哈','').replace('預備備','').replace('法文','').replace('獻聲','').replace('女聲','').replace('謝馨儀','').replace('夏宇','').replace('蘇打綠','').replace('周杰倫','').replace('開始瞜','').replace('很好聽喔','').replace('韓文','').replace('翻譯','').replace('中文','')
                lyrics = lyrics.replace('給一點掌聲好不好來','').replace('希望大家可以支持一下我的師妹袁詠琳再一次掌聲一下','').replace('日文','')
                if lyrics =='': #假設這首是全英文歌詞，那刪到最後就是空格，就不理他
                    continue
                if lyrics not in lyrics_lst:
                    lyrics_lst.append(lyrics)
            # time.sleep(0.5)  #不知道要設多久，我先設0.5秒
        f.write(str(lyrics_lst))

#文字雲
lyrics = open('singers_lyric.txt','r').read()
seg_list = jieba.cut(lyrics)
words = " ".join(seg_list)

font = 'TaipeiSansTCBeta-Regular.ttf'
mask = np.array(Image.open('picture.png'))
my_wordcloud = WordCloud(background_color='white',mask=mask,font_path=font).generate(words)


plt.imshow(my_wordcloud)
plt.axis("off")
plt.show()

my_wordcloud.to_file('word_cloud.png')

#歌詞產生器        
with open('singers_lyric.txt', 'r', encoding="utf-8") as f:
    text = f.read()

seg_lst = pseg.cut(text)
flag_lst = []
for i in seg_lst:
    flag_lst.append((i.word, i.flag)) #分詞類 #https://gist.github.com/hscspring/c985355e0814f01437eaf8fd55fd7998
# print(flag_lst)

first_lst = [] #存名詞、代詞
second_lst = [] # 存動詞、助詞
third_lst = [] # 存形容詞

for j in flag_lst: 
    if 'n' in j[1]: #名詞
        first_lst.append(j[0])
    if 'v' in j[1]: #動詞
        second_lst.append(j[0])
    if 'a' in j[1]: #形容詞
        third_lst.append(j[0])


no_repeat_first = sorted(set(first_lst)) #去重複
no_repeat_second= sorted(set(second_lst))
no_repeat_third= sorted(set(third_lst))

# print(no_repeat_first)
# print(no_repeat_second)
# print(no_repeat_third)

if text == '':
    print('對不起！查不到這位歌手喔！')

else:

    for j in range(2):#跑兩段
        for _ in range(8): #跑8行
            first = random.sample(no_repeat_first, 1) #隨機跑字 #名詞
            second = random.sample(no_repeat_second, 1) #動詞
            third = random.sample(no_repeat_third, 1) #形容詞
            fourth = random.sample(no_repeat_first, 1) #名詞

            print(''.join(first) + ''.join(second)+ ''.join(third)+ ''.join(fourth))
        print('')

endtime = datetime.datetime.now()
print ((endtime - starttime).seconds)

請輸入歌手名稱：蘇打綠


KeyboardInterrupt: ignored